# Week 3: Small Worlds — Assignment

**Learning objectives** — In this assignment you will:

- Implement sampled average path length using BFS
- Compute the small-world sigma coefficient
- Generate Watts-Strogatz graphs and verify their properties
- Sweep over rewiring probabilities and collect C/L statistics
- Generate ER random graphs
- Implement greedy routing on Kleinberg grids

## Grading

| Section | Function | Points |
|---------|----------|--------|
| 1 | `average_path_length(G, sample_size)` | 10 |
| 2 | `small_world_sigma(G, n_random)` | 20 |
| 3 | `generate_ws(n, k, p)` | 15 |
| 4 | `ws_sweep(n, k, p_values)` | 20 |
| 5 | `er_graph(n, avg_degree)` | 10 |
| 6 | `greedy_route(G, source, target, pos)` | 10 |
| — | Written questions | 15 |
| | **Total** | **100** |

## Before You Start

This assignment builds on the Week 3 lab. Make sure you are comfortable with:

- **Average shortest path length (APL)** — computed via BFS from every node; measures typical separation (Lab Section 2)
- **Small-world property** — high clustering AND short paths relative to a random baseline (Lab Section 3)
- **Watts-Strogatz model** — ring lattice + rewiring; the C(p)/L(p) vs p curve and its asymmetry (Lab Sections 4-5)
- **ER model** — random graph with fixed edge probability; serves as a null model for comparison (Lab Section 3)
- **Kleinberg model** — grid with distance-dependent long-range links; enables efficient decentralized search (Lab Section 7)

Sections 1-2 ask you to implement APL and sigma from scratch. Section 4 reproduces the classic WS sweep — review the lab's log-scale plot before starting.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from netsci.loaders import load_graph
from netsci.utils import SEED
from netsci import models

In [ ]:
G_karate = load_graph("karate")
G_air = load_graph("airports")

---
## Section 1: Average Path Length (10 pts)

Computing the exact average path length requires BFS from every node — O(n(n+m)) which can be slow on large graphs.

Implement a **sampled** version: pick `sample_size` random source nodes, run BFS from each,
and average all the distances.

The graph must be connected. Use `seed=SEED` for the random sampling.

In [ ]:
def average_path_length(G, sample_size=None):
    """Compute the (sampled) average shortest path length.

    Parameters
    ----------
    G : nx.Graph
        Must be connected.
    sample_size : int or None
        If None, use all nodes (exact). Otherwise sample this many
        source nodes using seed=SEED.

    Returns
    -------
    float
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
# Exact computation on karate should match NetworkX
_apl = average_path_length(G_karate, sample_size=None)
_expected = nx.average_shortest_path_length(G_karate)
assert abs(_apl - _expected) < 0.05, f"Got {_apl}, expected {_expected}"
print(f"Karate APL (exact): {_apl:.4f} (expected: {_expected:.4f})")

# Sampled should be close
_apl_sampled = average_path_length(G_air, sample_size=50)
_apl_exact = nx.average_shortest_path_length(G_air)
assert abs(_apl_sampled - _apl_exact) < 0.3, (
    f"Sampled {_apl_sampled} too far from exact {_apl_exact}"
)
print(f"Airports APL (sampled 50): {_apl_sampled:.2f} (exact: {_apl_exact:.2f})")
print("Section 1 passed!")

---
## Section 2: Small-World Sigma (20 pts)

Compute the small-world coefficient:

$$\sigma = \frac{C_{real} / C_{random}}{L_{real} / L_{random}}$$

Generate `n_random` Erdos-Renyi graphs with the same n and edge probability,
compute their average C and L, and use those as the baseline.

Use `SEED + i` as the seed for the i-th random graph to get different (but reproducible) baselines.

In [ ]:
def small_world_sigma(G, n_random=5):
    """Compute the small-world sigma coefficient.

    Parameters
    ----------
    G : nx.Graph
        Must be connected.
    n_random : int
        Number of random graphs to average over.

    Returns
    -------
    float
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
_sigma = small_world_sigma(G_air, n_random=5)
assert isinstance(_sigma, float)
# Airports should be a small world
assert _sigma > 1.0, f"Expected sigma > 1 for airports, got {_sigma}"
print(f"Airports sigma = {_sigma:.2f} (> 1 = small world)")
print("Section 2 passed!")

## Section 3: Watts-Strogatz Generation (15 pts)

Generate a connected Watts-Strogatz graph. Use `seed=SEED`.

Hint: use `models.watts_strogatz` (or equivalently `nx.connected_watts_strogatz_graph`).

In [ ]:
def generate_ws(n, k, p):
    """Generate a connected Watts-Strogatz graph.

    Parameters
    ----------
    n : int  — number of nodes
    k : int  — each node connected to k nearest neighbors
    p : float — rewiring probability

    Returns
    -------
    nx.Graph
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
_ws = generate_ws(100, 6, 0.1)
assert isinstance(_ws, nx.Graph)
assert _ws.number_of_nodes() == 100
assert nx.is_connected(_ws)

# p=0 should give a regular graph where every node has degree k
_ws0 = generate_ws(50, 4, 0.0)
_degrees = [d for _, d in _ws0.degree()]
assert all(d == 4 for d in _degrees), "p=0 should give uniform degree k"
print("Section 3 passed!")

---
## Section 4: WS Sweep (20 pts)

Sweep over a list of rewiring probabilities and collect the average clustering coefficient
and average shortest path length at each p.

Return a dictionary with keys:
- `"p"` — list of p values
- `"clustering"` — list of average clustering coefficients
- `"path_length"` — list of average shortest path lengths

In [ ]:
def ws_sweep(n, k, p_values):
    """Sweep over p values and collect WS graph statistics.

    Parameters
    ----------
    n : int
    k : int
    p_values : list of float

    Returns
    -------
    dict with keys 'p', 'clustering', 'path_length'
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
_ps = [0.0, 0.01, 0.1, 1.0]
_result = ws_sweep(100, 6, _ps)

assert set(_result.keys()) == {"p", "clustering", "path_length"}
assert len(_result["p"]) == 4
assert len(_result["clustering"]) == 4
assert len(_result["path_length"]) == 4

# Clustering should decrease with p
assert _result["clustering"][0] > _result["clustering"][-1], (
    "Clustering should be higher at p=0 than p=1"
)

# Path length should decrease with p
assert _result["path_length"][0] > _result["path_length"][-1], (
    "Path length should be higher at p=0 than p=1"
)

print(f"p=0.0:  C={_result['clustering'][0]:.3f}  L={_result['path_length'][0]:.2f}")
print(f"p=1.0:  C={_result['clustering'][-1]:.3f}  L={_result['path_length'][-1]:.2f}")
print("Section 4 passed!")

---
## Section 5: ER Graph (10 pts)

Generate an Erdos-Renyi graph with *n* nodes and a target average degree.
Compute the edge probability as `p = avg_degree / (n - 1)`. Use `seed=SEED`.

In [ ]:
def er_graph(n, avg_degree):
    """Generate an ER random graph.

    Parameters
    ----------
    n : int
    avg_degree : float

    Returns
    -------
    nx.Graph
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
_g = er_graph(500, 6)
assert isinstance(_g, nx.Graph)
assert _g.number_of_nodes() == 500
_avg_deg = 2 * _g.number_of_edges() / _g.number_of_nodes()
assert abs(_avg_deg - 6) < 1.5, f"Avg degree {_avg_deg} too far from 6"
print(f"ER(500, 6): actual avg degree = {_avg_deg:.2f}")
print("Section 5 passed!")

## Section 6: Greedy Routing on Kleinberg Grid (10 pts)

Implement greedy routing on a Kleinberg grid. At each step, forward the message to the
neighbor that is **closest to the target** by Manhattan distance.

Keep track of **visited nodes** to avoid cycles — do not revisit a node already on the path.

The function receives a graph `G` whose nodes are `(row, col)` tuples, a `source` node,
a `target` node, and a `pos` dict mapping nodes to coordinates.

Return the path as a list of nodes from source to target (inclusive), or `None` if the
current node has no unvisited neighbor closer to the target than itself (i.e., stuck in a local minimum).

In [ ]:
def greedy_route(G, source, target, pos):
    """Greedy routing on a grid graph.

    Parameters
    ----------
    G : nx.Graph — nodes are (row, col) tuples
    source : tuple — start node
    target : tuple — destination node
    pos : dict — {node: (row, col)} coordinates

    Returns
    -------
    list of nodes (path from source to target) or None if stuck
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# --- Validation ---
_G_k, _pos_k = models.kleinberg_grid(n=10, r=2, q=1, seed=SEED)
_nodes_k = list(_G_k.nodes())
_path = greedy_route(_G_k, _nodes_k[0], _nodes_k[-1], _pos_k)
assert _path is not None, "Greedy route should find a path on r=2 grid"
assert isinstance(_path, list)
assert _path[0] == _nodes_k[0], "Path should start at source"
assert _path[-1] == _nodes_k[-1], "Path should end at target"
# Path should be shorter than grid diameter
assert len(_path) < _G_k.number_of_nodes(), "Path too long"
# Each step should be along an edge
for i in range(len(_path) - 1):
    assert _G_k.has_edge(_path[i], _path[i + 1]), (
        f"No edge between {_path[i]} and {_path[i + 1]}"
    )
print(f"Greedy path length: {len(_path) - 1} hops")
print("Section 6 passed!")

---
## Written Questions (15 pts)

### Question 1 (5 pts)

In the Watts-Strogatz model, why does a **tiny amount of rewiring** (e.g., p=0.01) dramatically
reduce the average path length, but barely affect the clustering coefficient?
Think about what a single long-range shortcut does to distances in a ring lattice.

*Hints to guide your thinking:*
- *In a ring lattice, the shortest path between opposite nodes must traverse half the ring — roughly n/(2k) hops. What happens when you add a single "shortcut" across the ring?*
- *One rewired edge creates a shortcut that benefits not just its two endpoints, but all pairs whose shortest path now detours through it. How many pairs benefit from a single shortcut?*
- *Clustering, on the other hand, is a **local** property — it only changes if a triangle is created or destroyed. How many triangles does one rewired edge affect?*

**Your Answer:**



### Question 2 (4 pts)

The airports network has sigma > 1, confirming it is a small world.
Does this mean "six degrees of separation" literally holds for airports?
What is the actual average path length, and what would "six degrees" mean in this context?

*Hints to guide your thinking:*
- *"Six degrees of separation" refers to social networks with billions of people. The airports network has 500 nodes — what was its actual average path length from the lab?*
- *Sigma > 1 confirms the structural property (high C, short L relative to random), but the absolute path length depends on network size.*
- *In the airport context, "degrees" are layovers. Is 2-3 layovers between any pair of airports surprising? What makes this possible?*

**Your Answer:**



### Question 3 (6 pts)

Why does r=0 fail for greedy routing even though the graph has short paths? What property of r=2 links makes them useful for decentralized search?

*Hints to guide your thinking:*
- *At r=0, long-range links are uniform random. When you're at node A trying to reach node B, how likely is it that any of your long-range neighbors are actually closer to B than you are?*
- *At r=2, long-range links follow P ∝ d^(-2). This means you have links at many different distance scales. Why is having links at every scale important for greedy search?*
- *Think about the journey in stages: when you're far from the target, you need a long-range link. When you're close, you need a medium-range link. What distribution of link lengths ensures you always have a useful shortcut?*

**Your Answer:**

